In [68]:
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector
from mesa import Agent, Model 
from mesa.space import MultiGrid


%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128
import datetime
import signal
import math
import numpy as np
import pandas as pd
import time


In [74]:
stackedBoxes = 0
numberOfBoxes = 0
numberOfStacks = 0
 
numStack = 0
boxesDepositedInCurrentStack = 0


def startCalculations(K):
  global numberOfBoxes
  global numberOfStacks
 
  numberOfBoxes = K
  numberOfStacks = math.floor(K/5)
 



def get_grid(model):
    
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
            grid[x][y] = agent.height
    return grid



class Box(Agent):

    def __init__(self, unique_id, model, height):
      super().__init__(unique_id, model)
      self.height = height
      self.stacked = False
      
class Robot(Agent):

    def __init__(self, unique_id, model, height):
      super().__init__(unique_id, model)
      self.height = height
      self.hasBox = 0
      self.box = -1

    def moveToStack(self):
      global numStack
      x, y = self.pos
      if x != 0 and self.canIMoveThere((x-1,y)):
        self.model.grid.move_agent(self, (x-1,y))
      if y < numStack and self.canIMoveThere((x,y+1)):
        self.model.grid.move_agent(self, (x,y+1))
      if y > numStack and self.canIMoveThere((x,y-1)):
        self.model.grid.move_agent(self, (x,y-1))
      if x==0 and y==numStack:
        self.depositBox()

    


    def canIMoveThere(self, pos):
      neighbours = self.model.grid.get_neighbors(
          self.pos,
          moore=False,
          include_center=False)
      for neighbour in neighbours:
        if (isinstance(neighbour, Robot) and neighbour.pos == pos):
          return False
      return True         

    def findBox(self):
      neighbours = self.model.grid.get_neighbors(
          self.pos,
          moore=False,
          include_center=True)
      for neighbour in neighbours:
        if (isinstance(neighbour, Box)):
          if (neighbour.height) == 1 and neighbour.stacked == False:
            return neighbour
      return -1 
          
    def depositBox(self):
      global boxesDepositedInCurrentStack
      global stackedBoxes
      global numStack
      b = Box(self.pos, self, boxesDepositedInCurrentStack + 1)
      b.stacked = True
      self.model.grid.place_agent(b, self.pos)
      boxesDepositedInCurrentStack +=1
      self.hasBox = 2
      stackedBoxes +=1
      if boxesDepositedInCurrentStack == 5:
        numStack +=1
        boxesDepositedInCurrentStack = 0
        
    def moveFromStack(self) :
      x, y = self.pos
      if x != self.model.width/2 and self.canIMoveThere((x+1,y)):
        self.model.grid.move_agent(self, (x+1,y))
      if y != self.model.height/2 and self.canIMoveThere((x,y+1)):
        self.model.grid.move_agent(self, (x,y+1))
      if x==self.model.width/2 and y==self.model.height/2:
        self.hasBox = 0  



    def random_move(self):
      next_moves = self.model.grid.get_neighborhood(self.pos, False, True)
      next_move = self.random.choice(next_moves)
      # Realizar el movimiento
      xNext, yNext = next_move
      x, y = self.pos

      if self.canIMoveThere(next_move):
        if (x == 0 and xNext == self.model.height - 1):
          return
        if (y == 0 and yNext == self.model.width - 1):
          return
        if (x == self.model.height - 1 and xNext == 0):
          return
        if (y == self.model.width - 1 and yNext == 0):
          return    
        self.model.grid.move_agent(self, next_move)



    def step(self):
      if self.hasBox == 0:
        self.box = self.findBox()
        if self.box != -1:             
          self.hasBox = 1
          self.model.grid.remove_agent(self.box)
        

    def advance(self):
      if self.hasBox == 1:
        self.moveToStack()
      elif self.hasBox == 2:
        self.moveFromStack()
      else:
        self.random_move()


class RobotModel(Model):
    
    def __init__(self, width, height, K):
      self.grid = MultiGrid(width, height, False)
      self.schedule = SimultaneousActivation(self)
      self.numBoxes = K
      self.width = width
      self.height = height
      global numberOfBoxes
      startCalculations(K)
      gridSize = width*height
      
      arrayPositions = np.zeros((width,height), int)
      x = 0
      while x < K:
        pos = np.random.randint(0,width-1)
        pos2 = np.random.randint(0,height-1)
        if arrayPositions[pos][pos2] == 0:
          arrayPositions[pos][pos2] = 1
          x+=1

      y = 0
      while y < 5:
        pos = np.random.randint(0,width-1)
        pos2 = np.random.randint(0,height-1)
        if arrayPositions[pos][pos2] == 0:
          arrayPositions[pos][pos2] = 2
          y+=1
      
      for (content, x, y) in self.grid.coord_iter():
        if arrayPositions[x][y] == 2:
          a = Robot((x, y), self, 6)
          self.grid.place_agent(a, (x,y)) 
          self.schedule.add(a)
        if arrayPositions[x][y] == 1:
          b = Box((x, y), self, 1)
          self.grid.place_agent(b, (x,y))
          self.schedule.add(b)


      self.datacollector = DataCollector(
          model_reporters={"Grid": get_grid})
    
    def allStacked(self):
      self.percentageStacked = stackedBoxes / self.numBoxes
      if self.percentageStacked == 1 :
        return 1
      else:
        return self.percentageStacked



    def step(self):
        
      self.datacollector.collect(self)
      self.schedule.step()

In [75]:
GRID_SIZE = 10

K = 25


start_time = time.time()
maxTime = .1

frameCounter = 0

model = RobotModel(GRID_SIZE, GRID_SIZE, K)

while(time.time() - start_time) < maxTime and model.allStacked() != 1:
  frameCounter+=1
  model.step()

frameCounter+=1
model.step()


print('Tiempo en segundos:', str(datetime.timedelta(seconds=(time.time() - start_time))), 'seg' )

print('Movs: ', frameCounter)


Tiempo en segundos: 0:00:00.038927 seg
Movs:  343


In [76]:
all_grid = model.datacollector.get_model_vars_dataframe()


In [77]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.nipy_spectral)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=frameCounter)

In [78]:

anim